---
### Set up the same stop criteria: “when there is no change in centroid position OR when the SSE value increases in the next iteration OR when the maximum preset value (e.g., 500, you can set the preset value by yourself) of iteration is complete”, for Euclidean-K-means, Cosine-K- means, Jarcard-K-means. Which method requires more iterations and times to converge? (10 points)
---

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import time
import random

In [2]:
data = pd.read_csv("/content/data.csv",header=None).values
label = pd.read_csv("/content/label.csv",header=None).values.reshape(-1)

Define distances/similarities 

In [3]:
def euclidean_dist(a, b):
    return np.linalg.norm(a - b)

def cosine_sim(a, b):
    return 1 - np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def jaccard_sim(a, b):
    return 1 - np.sum(np.minimum(a, b)) / np.sum(np.maximum(a, b))

In [4]:
k = len(np.unique(label))
k

10

**K-Means**

In [5]:
class KMeans:
    def __init__(self, k=10, max_iter=100, distance=euclidean_dist, seed=42):
        self.k = k
        self.max_iter = max_iter
        self.distance = distance
        self.seed = seed
    
    def fit(self, X):
        np.random.seed(self.seed)
        self.centroids = X[np.random.choice(X.shape[0], self.k, replace=False)]
        prev_sse = np.inf
        for i in range(self.max_iter):
            clusters = [[] for _ in range(self.k)]
            for x in X:
                distances = [self.distance(x, c) for c in self.centroids]
                cluster = np.argmin(distances)
                clusters[cluster].append(x)
            new_centroids = []
            for cluster in clusters:
                if len(cluster) == 0:
                    new_centroids.append(np.zeros(X.shape[1]))
                else:
                    new_centroids.append(np.mean(cluster, axis=0))
            sse = np.sum([np.sum((cluster - centroid)**2) for cluster, centroid in zip(clusters, new_centroids)])
            if np.allclose(self.centroids, new_centroids) or sse > prev_sse:
                break
            self.centroids = new_centroids
            prev_sse = sse
    
    def predict(self, X):
        distances = np.array([np.array([self.distance(x, c) for c in self.centroids]) for x in X])
        return np.argmin(distances, axis=1)

In [6]:
t1 = time.time()

k_euclid = KMeans(k=10, max_iter=500, distance=euclidean_dist)
k_euclid.fit(data)
t2 = time.time()


In [7]:
t3 = time.time()

k_cosine = KMeans(k=10, max_iter=500, distance=cosine_sim)
k_cosine.fit(data)
t4 = time.time()


In [8]:
t5 = time.time()
k_jacard = KMeans(k=10, max_iter=500, distance=jaccard_sim)
k_jacard.fit(data)
t6 = time.time()


In [9]:
print("Euclidean converged in {} iterations".format(len(k_euclid.centroids)))
print("Time by Euclidean to converge {}\n".format(t2-t1))

print("Cosine converged in {} iterations".format(len(k_cosine.centroids)))
print("Time by Cosine to converge {}\n".format(t4-t3))

print("Jaccard converged in {} iterations".format(len(k_jacard.centroids)))
print("Time by Jaccard to converge {}\n".format(t6-t5))

Euclidean converged in 10 iterations
Time by Euclidean to converge 25.6650710105896

Cosine converged in 10 iterations
Time by Cosine to converge 44.05938649177551

Jaccard converged in 10 iterations
Time by Jaccard to converge 61.64747166633606



**Comment:**

All methods required the same amount of iterations.

Euclidian took less time to converage

---
### Compare the SSEs of Euclidean-K-means Cosine-K-means, Jarcard-K-means with respect to the following three terminating conditions: (10 points)
* when there is no change in centroid position
* when the SSE value increases in the next iteration
* when the maximum preset value (e.g., 100) of iteration is complete
---

* **when there is no change in centroid position**

In [10]:

class KMeans:
    def __init__(self, k=10, max_iter=100, distance=euclidean_dist, seed=42):
        self.k = k
        self.max_iter = max_iter
        self.distance = distance
        self.seed = seed

    def fit(self, X):
        np.random.seed(self.seed)
        self.centroids = X[np.random.choice(X.shape[0], self.k, replace=False)]
        prev_centroids = None
        for i in range(self.max_iter):
            clusters = [[] for _ in range(self.k)]
            for x in X:
                distances = cdist(x.reshape(1, -1), self.centroids, metric=self.distance)
                cluster = np.argmin(distances)
                clusters[cluster].append(x)
            prev_centroids = self.centroids.copy()
            self.centroids = [np.mean(cluster, axis=0) for cluster in clusters]
            if np.allclose(prev_centroids, self.centroids):
                break

    def predict(self, X):
        distances = cdist(X, self.centroids, metric=self.distance)
        return np.argmin(distances, axis=1)

# Sample data
X, y = data,label

# Models
euclidean_kmeans = KMeans(k=10, max_iter=100, distance=euclidean_dist, seed=42)
cosine_kmeans = KMeans(k=10, max_iter=100, distance=cosine_sim, seed=42)
jaccard_kmeans = KMeans(k=10, max_iter=100, distance=jaccard_sim, seed=42)

In [11]:
euclidean_kmeans.fit(X)
euclidean_sse = np.sum(np.min(cdist(X, euclidean_kmeans.centroids, 'euclidean'), axis=1)**2)


In [12]:
cosine_kmeans.fit(X)
cosine_sse = np.sum(np.min(cdist(X, cosine_kmeans.centroids, 'cosine'), axis=1)**2)


In [13]:
jaccard_kmeans.fit(X)
jaccard_sse = np.sum(np.min(cdist(X, jaccard_kmeans.centroids, 'jaccard'), axis=1)**2)


In [14]:
print("Euclidean SSE:\n", euclidean_sse)
print("Cosine SSE:\n", cosine_sse)
print("Jaccard SSE:\n", jaccard_sse)

Euclidean SSE:
 25414767689.961178
Cosine SSE:
 686.4355725684936
Jaccard SSE:
 9999.83773073805


* **when the SSE value increases in the next iteration**

In [15]:
class KMeans:
    def __init__(self, k=10, max_iter=100, distance=euclidean_dist, seed=42):
        self.k = k
        self.max_iter = max_iter
        self.distance = distance
        self.seed = seed

    def fit(self, X):
        np.random.seed(self.seed)
        self.centroids = X[np.random.choice(X.shape[0], self.k, replace=False)]
        prev_centroids = None
        prev_sse = float('inf')
        for i in range(self.max_iter):
            clusters = [[] for _ in range(self.k)]
            for x in X:
                distances = cdist(x.reshape(1, -1), self.centroids, metric=self.distance)
                cluster = np.argmin(distances)
                clusters[cluster].append(x)
            prev_centroids = self.centroids.copy()
            self.centroids = [np.mean(cluster, axis=0) for cluster in clusters]
            sse = np.sum([np.sum(cdist(cluster, centroid.reshape(1,-1), metric=self.distance)**2) for cluster, centroid in zip(clusters, self.centroids)])
            if sse > prev_sse:
                break
            prev_sse = sse
            if np.allclose(prev_centroids, self.centroids):
                break

    def predict(self, X):
        distances = cdist(X, self.centroids, metric=self.distance)
        return np.argmin(distances, axis=1)

# Data
X, y = data,label

# Models
euclidean_kmeans = KMeans(k=10, max_iter=100, distance=euclidean_dist, seed=42)
cosine_kmeans = KMeans(k=10, max_iter=100, distance=cosine_sim, seed=42)
jaccard_kmeans = KMeans(k=10, max_iter=100, distance=jaccard_sim, seed=42)

# Fit Model
euclidean_kmeans.fit(X)
cosine_kmeans.fit(X)
jaccard_kmeans.fit(X)

euclidean_sse = np.sum(np.min(cdist(X, euclidean_kmeans.centroids, 'euclidean'), axis=1)**2)
cosine_sse = np.sum(np.min(cdist(X, cosine_kmeans.centroids, 'cosine'), axis=1)**2)
jaccard_sse = np.sum(np.min(cdist(X, jaccard_kmeans.centroids, 'jaccard'), axis=1)**2)

# Print SSE
print("Euclidean SSE:\n", euclidean_sse)
print("Cosine SSE:\n", cosine_sse)
print("Jaccard SSE:\n", jaccard_sse)

Euclidean SSE:
 25414767689.961178
Cosine SSE:
 686.229294287177
Jaccard SSE:
 9999.907741263536


* **when the maximum preset value (e.g., 100) of iteration is complete**

In [16]:
# Define KMeans class
class KMeans:
    def __init__(self, k=10, max_iter=100, distance=euclidean_dist, seed=42):
        self.k = k
        self.max_iter = max_iter
        self.distance = distance
        self.seed = seed

    def fit(self, X):
        np.random.seed(self.seed)
        self.centroids = X[np.random.choice(X.shape[0], self.k, replace=False)]
        prev_centroids = None
        for i in range(self.max_iter):
            clusters = [[] for _ in range(self.k)]
            for x in X:
                distances = cdist(x.reshape(1, -1), self.centroids, metric=self.distance)
                cluster = np.argmin(distances)
                clusters[cluster].append(x)
            prev_centroids = self.centroids.copy()
            self.centroids = [np.mean(cluster, axis=0) for cluster in clusters]
            if np.allclose(prev_centroids, self.centroids):
                break
            elif i == self.max_iter-1:
                print(f"KMeans did not converge in {self.max_iter} iterations")
                break

    def predict(self, X):
        distances = cdist(X, self.centroids, metric=self.distance)
        return np.argmin(distances, axis=1)

# Data
X, y = data,label

# Models
euclidean_kmeans = KMeans(k=10, max_iter=100, distance=euclidean_dist, seed=42)
cosine_kmeans = KMeans(k=10, max_iter=100, distance=cosine_sim, seed=42)
jaccard_kmeans = KMeans(k=10, max_iter=100, distance=jaccard_sim, seed=42)

# Fit Model
euclidean_kmeans.fit(X)
cosine_kmeans.fit(X)
jaccard_kmeans.fit(X)


euclidean_sse = sum(np.min(cdist(X, euclidean_kmeans.centroids, euclidean_dist), axis=1)**2)
cosine_sse = sum(np.min(cdist(X, cosine_kmeans.centroids, cosine_sim), axis=1)**2)
jaccard_sse = sum(np.min(cdist(X, jaccard_kmeans.centroids, jaccard_sim), axis=1)**2)

# Print SSE
print("Euclidean SSE:\n", euclidean_sse)
print("Cosine SSE:\n", cosine_sse)
print("Jaccard SSE:\n", jaccard_sse)

Euclidean SSE:
 25414767689.9611
Cosine SSE:
 686.435572568491
Jaccard SSE:
 3660.389493716567


---
###What are your summary observations or takeaways based on your algorithmic analysis? (5 points)
---

**Comment:**

The summary oberservation for all questions/tasks (1,2,3,4,5)

For Questions 1 and 2

It seems that Cosine Similarity is the better method against the other distances when we run a regular K-means 

For Questins 3,4,5

All Methods seem to converage at the same iteration but Euclid being the faster to do so.

Now, when we apply different stop criteria. For the first 2 conditions there seems to be no much of a difference

But when it comes to applying the max present value of iteration, Cosine performs better than the other two. 

This can also be based on the type of data. 
